<a href="https://colab.research.google.com/github/binhluong84/Machine-Learning/blob/main/Dandelion_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Connect Google Drive
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import tensorflow as tf

from sklearn.metrics import accuracy_score, f1_score

In [ ]:
image_dir = Path('/gdrive/MyDrive/Images')

In [ ]:
filepaths = list(image_dir.glob(r'**/*.jpg'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')

image_df = pd.concat([filepaths, labels], axis=1)

train_df, test_df = train_test_split(image_df, train_size=0.7, shuffle=True, random_state=1)

In [ ]:
train_df

,Filepath,Label
491,/gdrive/MyDrive/Images/dandelion/IMG_6117.jpg,dandelion
1091,/gdrive/MyDrive/Images/other/IMG_6236.jpg,other
512,/gdrive/MyDrive/Images/dandelion/IMG_6136.jpg,dandelion
919,/gdrive/MyDrive/Images/other/IMG_5869.jpg,other
741,/gdrive/MyDrive/Images/other/IMG_5648.jpg,other
...,...,...
715,/gdrive/MyDrive/Images/other/IMG_5618.jpg,other
905,/gdrive/MyDrive/Images/other/IMG_5857.jpg,other
1096,/gdrive/MyDrive/Images/other/IMG_6243.jpg,other
235,/gdrive/MyDrive/Images/dandelion/IMG_3884.jpg,dandelion


In [ ]:
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
    validation_split=0.2
)

test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
)

In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=32,
    shuffle=False
)

Found 707 validated image filenames belonging to 2 classes.
Found 176 validated image filenames belonging to 2 classes.
Found 379 validated image filenames belonging to 2 classes.


In [ ]:
feature_extractor = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    weights='imagenet',
    include_top=False,
    pooling='avg'
)

feature_extractor.trainable = False

9406464/9406464 [==============================] - 0s 0us/step


In [ ]:
inputs = feature_extractor.input
x = tf.keras.layers.Dense(128, activation='relu')(feature_extractor.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 32  128         ['Conv1[0][0]']                  
                                )                                                             

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/100
23/23 [==============================] - 454s 20s/step - loss: 0.5958 - accuracy: 0.6733 - val_loss: 0.4490 - val_accuracy: 0.7614
Epoch 2/100
23/23 [==============================] - 115s 5s/step - loss: 0.3746 - accuracy: 0.8190 - val_loss: 0.4336 - val_accuracy: 0.7955
Epoch 3/100
23/23 [==============================] - 118s 5s/step - loss: 0.3213 - accuracy: 0.8515 - val_loss: 0.3989 - val_accuracy: 0.8011
Epoch 4/100
23/23 [==============================] - 118s 5s/step - loss: 0.2592 - accuracy: 0.8868 - val_loss: 0.3430 - val_accuracy: 0.8523
Epoch 5/100
23/23 [==============================] - 114s 5s/step - loss: 0.2221 - accuracy: 0.9081 - val_loss: 0.3221 - val_accuracy: 0.8807
Epoch 6/100
23/23 [==============================] - 115s 5s/step - loss: 0.1623 - accuracy: 0.9392 - val_loss: 0.3917 - val_accuracy: 0.8409
Epoch 7/100
23/23 [==============================] - 116s 5s/step - loss: 0.1535 - accuracy: 0.9321 - val_loss: 0.4104 - val_accuracy: 0.8409
Epoch

In [ ]:
predictions = np.squeeze(model.predict(test_images))
predictions = (predictions >= 0.5).astype(np.int)

acc = accuracy_score(test_images.labels, predictions)
f1 = f1_score(test_images.labels, predictions)

print("Accuracy: {:.2f}%".format(acc * 100))
print("F1-Score: {:.5f}".format(f1))

12/12 [==============================] - 188s 17s/step
Accuracy: 83.64%
F1-Score: 0.84729


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  
